# Import libraries

In [1973]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ast

# Set global data path

In [1974]:
# path where the csv files are saved
data_folder = '/Users/luisalcala/Desktop/redbull/data'

# Load raw data

In [1975]:
# dictionaty to store the standings
standings_files = {}

# for each file in the path (we only have csv files)
for file_name in os.listdir(data_folder):
    print(file_name)
    if file_name[0] == 's':
        # read csv with particular header
        df = pd.read_csv(os.path.join(data_folder, file_name), encoding='ISO-8859-1')
        # Add the DataFrame to the dictionary with key = file name
        standings_files[os.path.splitext(file_name)[0]] = df
    elif file_name[0] == 't':
        # read transfer csv
        transfers_df = pd.read_csv(os.path.join(data_folder, file_name), index_col=[0])

standings-bundesliga-19-20.csv
clean_full_standings.csv
standings-bundesliga-21-22.csv
transfers-bundesliga.csv
standings-bundesliga-23-24.csv
clean_transfers.csv
standings-bundesliga-22-23.csv
standings-bundesliga-20-21.csv


In [1976]:
len(standings_files)

5

In [1977]:
transfers_df.head()

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_euro,transfer_fee_million,...,to.club_name,to.competition_id,to.competition_name,date_of_birth_datetime,citizenship,position.main,position.other,height_cm,foot,updated_at
0,2518704,2019-07-01,free loan transfer,19/20,2019,314245,Emmanuel Iyoha,21.0,NaN,NaN,...,Holstein Kiel,L2,2. Bundesliga,1997-10-11,['Germany' 'Nigeria'],Left-Back,['Right-Back' 'Left Winger'],191.0,right,2024-02-25T02:16:17.122733
1,2472611,2019-07-01,free transfer,19/20,2019,153634,Marcel Hilßner,24.0,0.0,0.0,...,SC Paderborn,L1,Bundesliga,1995-01-30,['Germany'],Attacking Midfield,['Right Winger' 'Left Winger'],183.0,left,2024-02-24T10:13:57.036699
2,2512548,2019-07-01,free loan transfer,19/20,2019,195676,Ole Käuper,22.0,NaN,NaN,...,Carl Zeiss Jena,L3,3. Liga,1997-01-09,['Germany'],Defensive Midfield,['Central Midfield'],187.0,left,2024-02-24T10:13:48.186890
3,2507880,2019-07-01,free loan transfer,19/20,2019,236986,Thore Jacobsen,22.0,NaN,NaN,...,Magdeburg,L3,3. Liga,1997-04-19,['Germany'],Defensive Midfield,['Central Midfield' 'Left-Back'],182.0,left,2024-02-24T10:13:43.016587
4,2491956,2019-07-01,NaN,19/20,2019,31652,Fabian Schönheim,32.0,NaN,NaN,...,Without Club,NaN,NaN,1987-02-14,['Germany'],Centre-Back,['Left-Back' 'Defensive Midfield'],NaN,NaN,2024-02-24T23:49:23.092466


In [1978]:
standings_files['standings-bundesliga-19-20'].head()

,Unnamed: 0,Unnamed: 1,Home,Unnamed: 3,Unnamed: 4,Unnamed: 5,Away,Unnamed: 7,Unnamed: 8,Unnamed: 9,Total,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,NaN,NaN,P,W,D,L,P,W,D,L,P,W,D,L,F,A,+/-,Pts
1,C,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,+68,82
2,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,+43,69
3,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,+44,66
4,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,+26,65


# Standings Preprocessing

## Process column names

In [1979]:
# as all stadings files have the same structure, we can set the column names we'd like
column_names = [
    'final_standing', 'team_name', 
    'home_played', 'home_wins', 'home_draws', 'home_losses', 
    'away_played', 'away_wins', 'away_draws', 'away_losses', 
    'total_played', 'total_wins', 'total_draws', 'total_losses',
    'goals_in_favor', 'goals_against', 'goals_balance', 'points'
    ]

# change column names for the standings files
for file_name, df in standings_files.items():
    df.columns = column_names
    df.drop(index=df.index[0], inplace=True)
    df.reset_index(drop=True, inplace=True)

In [1980]:
standings_files['standings-bundesliga-19-20'].head(18)

,final_standing,team_name,home_played,home_wins,home_draws,home_losses,away_played,away_wins,away_draws,away_losses,total_played,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points
0,C,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,+68,82
1,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,+43,69
2,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,+44,66
3,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,+26,65
4,5,Bayer 04 Leverkusen,17,9,4,4,17,10,2,5,34,19,6,9,61,44,+17,63
5,6,1899 Hoffenheim,17,7,1,9,17,8,6,3,34,15,7,12,53,53,0,52
6,7,VfL Wolfsburg,17,4,7,6,17,9,3,5,34,13,10,11,48,46,+2,49
7,8,SC Freiburg,17,9,2,6,17,4,7,6,34,13,9,12,48,47,+1,48
8,9,Eintracht Frankfurt,17,8,4,5,17,5,2,10,34,13,6,15,59,60,-1,45
9,10,Hertha Berlin,17,6,3,8,17,5,5,7,34,11,8,15,48,59,-11,41


In [1981]:
standings_files['standings-bundesliga-20-21'].head(18)

,final_standing,team_name,home_played,home_wins,home_draws,home_losses,away_played,away_wins,away_draws,away_losses,total_played,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points
0,C,Bayern Munich,17,13,4,0,17,11,2,4,34,24,6,4,99,44,+55,78
1,2,RB Leipzig,17,11,4,2,17,8,4,5,34,19,8,7,60,32,+28,65
2,3,Borussia Dortmund,17,11,2,4,17,9,2,6,34,20,4,10,75,46,+29,64
3,4,VfL Wolfsburg,17,10,4,3,17,7,6,4,34,17,10,7,61,37,+24,61
4,5,Eintracht Frankfurt,17,10,7,0,17,6,5,6,34,16,12,6,69,53,+16,60
5,6,Bayer 04 Leverkusen,17,8,5,4,17,6,5,6,34,14,10,10,53,39,+14,52
6,7,1. FC Union Berlin,17,8,8,1,17,4,6,7,34,12,14,8,50,43,+7,50
7,8,Borussia Mönchengladbach,17,8,4,5,17,5,6,6,34,13,10,11,64,56,+8,49
8,9,VfB Stuttgart,17,5,6,6,17,7,3,7,34,12,9,13,56,55,+1,45
9,10,SC Freiburg,17,7,6,4,17,5,3,9,34,12,9,13,52,52,0,45


## Define constant vars

In [1982]:
N_TEAMS_SEASON = 18

## Change C and R values per corresponding numerical standing

In some files, the Champions and the Relegation teams are marked as C and R respectively, let's change that to 1 and 17/18 respectively. To do that, we just set the final standing as the index value +1 as we know the first is going to be the champion and the last two relegated (does not change over the seasons for now). 

In [1983]:
for file_name, df in standings_files.items():
    df['final_standing'] = df.index + 1

In [1984]:
standings_files['standings-bundesliga-19-20'].head(18)

,final_standing,team_name,home_played,home_wins,home_draws,home_losses,away_played,away_wins,away_draws,away_losses,total_played,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points
0,1,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,+68,82
1,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,+43,69
2,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,+44,66
3,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,+26,65
4,5,Bayer 04 Leverkusen,17,9,4,4,17,10,2,5,34,19,6,9,61,44,+17,63
5,6,1899 Hoffenheim,17,7,1,9,17,8,6,3,34,15,7,12,53,53,0,52
6,7,VfL Wolfsburg,17,4,7,6,17,9,3,5,34,13,10,11,48,46,+2,49
7,8,SC Freiburg,17,9,2,6,17,4,7,6,34,13,9,12,48,47,+1,48
8,9,Eintracht Frankfurt,17,8,4,5,17,5,2,10,34,13,6,15,59,60,-1,45
9,10,Hertha Berlin,17,6,3,8,17,5,5,7,34,11,8,15,48,59,-11,41


In [1985]:
standings_files['standings-bundesliga-20-21'].head(18)

,final_standing,team_name,home_played,home_wins,home_draws,home_losses,away_played,away_wins,away_draws,away_losses,total_played,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points
0,1,Bayern Munich,17,13,4,0,17,11,2,4,34,24,6,4,99,44,+55,78
1,2,RB Leipzig,17,11,4,2,17,8,4,5,34,19,8,7,60,32,+28,65
2,3,Borussia Dortmund,17,11,2,4,17,9,2,6,34,20,4,10,75,46,+29,64
3,4,VfL Wolfsburg,17,10,4,3,17,7,6,4,34,17,10,7,61,37,+24,61
4,5,Eintracht Frankfurt,17,10,7,0,17,6,5,6,34,16,12,6,69,53,+16,60
5,6,Bayer 04 Leverkusen,17,8,5,4,17,6,5,6,34,14,10,10,53,39,+14,52
6,7,1. FC Union Berlin,17,8,8,1,17,4,6,7,34,12,14,8,50,43,+7,50
7,8,Borussia Mönchengladbach,17,8,4,5,17,5,6,6,34,13,10,11,64,56,+8,49
8,9,VfB Stuttgart,17,5,6,6,17,7,3,7,34,12,9,13,56,55,+1,45
9,10,SC Freiburg,17,7,6,4,17,5,3,9,34,12,9,13,52,52,0,45


## Join in one dataframe by year

For each standing file, we set the season id based on the file name to match the season id of the transfers file in case we need to cross data later.

In [1986]:
standings_list = []

for file_name, df in standings_files.items():
    # set season id with the file name
    df['season_id'] = '20'+file_name[len(file_name)-5:len(file_name)-3]
    standings_list.append(df)

# concat all standings in a single dataframe
standings_df = pd.concat(standings_list, ignore_index=True)
print(standings_df.shape)
standings_df.head()

(90, 19)


,final_standing,team_name,home_played,home_wins,home_draws,home_losses,away_played,away_wins,away_draws,away_losses,total_played,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points,season_id
0,1,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,+68,82,2019
1,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,+43,69,2019
2,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,+44,66,2019
3,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,+26,65,2019
4,5,Bayer 04 Leverkusen,17,9,4,4,17,10,2,5,34,19,6,9,61,44,+17,63,2019


In [1987]:
standings_df.sort_values(by=['season_id', 'final_standing'], inplace=True)

## Take out + sign on goals

In [1988]:
standings_df['goals_balance'] = standings_df['goals_balance'].str.replace('+', '', regex=False)
standings_df.head()

,final_standing,team_name,home_played,home_wins,home_draws,home_losses,away_played,away_wins,away_draws,away_losses,total_played,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points,season_id
0,1,Bayern Munich,17,13,2,2,17,13,2,2,34,26,4,4,100,32,68,82,2019
1,2,Borussia Dortmund,17,11,3,3,17,10,3,4,34,21,6,7,84,41,43,69,2019
2,3,RB Leipzig,17,7,8,2,17,11,4,2,34,18,12,4,81,37,44,66,2019
3,4,Borussia Mönchengladbach,17,12,2,3,17,8,3,6,34,20,5,9,66,40,26,65,2019
4,5,Bayer 04 Leverkusen,17,9,4,4,17,10,2,5,34,19,6,9,61,44,17,63,2019


## Set correct data types

In [1989]:
standings_df.columns

Index(['final_standing', 'team_name', 'home_played', 'home_wins', 'home_draws',
       'home_losses', 'away_played', 'away_wins', 'away_draws', 'away_losses',
       'total_played', 'total_wins', 'total_draws', 'total_losses',
       'goals_in_favor', 'goals_against', 'goals_balance', 'points',
       'season_id'],
      dtype='object')

In [1990]:
standings_df.dtypes

final_standing     int64
team_name         object
home_played       object
home_wins         object
home_draws        object
home_losses       object
away_played       object
away_wins         object
away_draws        object
away_losses       object
total_played      object
total_wins        object
total_draws       object
total_losses      object
goals_in_favor    object
goals_against     object
goals_balance     object
points            object
season_id         object
dtype: object

In [1991]:
# set numerical columns
numerical_cols = ['final_standing', 'home_played', 'home_wins', 'home_draws',
       'home_losses', 'away_played', 'away_wins', 'away_draws', 'away_losses',
       'total_played', 'total_wins', 'total_draws', 'total_losses',
       'goals_in_favor', 'goals_against', 'goals_balance', 'points']

# set categorical columns
cat_cols = ['team_name', 'season_id']

In [1992]:
standings_df[numerical_cols] = standings_df[numerical_cols].apply(pd.to_numeric, errors='coerce')

In [1993]:
standings_df.dtypes

final_standing     int64
team_name         object
home_played        int64
home_wins          int64
home_draws         int64
home_losses        int64
away_played        int64
away_wins          int64
away_draws         int64
away_losses        int64
total_played       int64
total_wins         int64
total_draws        int64
total_losses       int64
goals_in_favor     int64
goals_against      int64
goals_balance      int64
points             int64
season_id         object
dtype: object

In [1994]:
standings_df.tail()

,final_standing,team_name,home_played,home_wins,home_draws,home_losses,away_played,away_wins,away_draws,away_losses,total_played,total_wins,total_draws,total_losses,goals_in_favor,goals_against,goals_balance,points,season_id
49,14,Borussia Mönchengladbach,17,5,6,6,17,2,7,8,34,7,13,14,56,67,-11,34,2023
50,15,1 FC Union Berlin,17,7,2,8,17,2,4,11,34,9,6,19,33,58,-25,33,2023
51,16,VfL Bochum,17,5,8,4,17,2,4,11,34,7,12,15,42,74,-32,33,2023
52,17,1 FC Köln,17,4,4,9,17,1,8,8,34,5,12,17,28,60,-32,27,2023
53,18,Darmstadt 98,17,1,3,13,17,2,5,10,34,3,8,23,30,86,-56,17,2023


## Clean team names

In [1995]:
d_team_name_correction = {
    '1. FC Union Berlin': 'FC Union Berlin',
    '1 FC Union Berlin': 'FC Union Berlin',
    '1. FC Köln': 'FC Köln',
    '1 FC Köln': 'FC Köln',
    '1 FC Heidenheim': 'FC Heidenheim'
}

standings_df['team_name'] = standings_df['team_name'].replace(d_team_name_correction)

## Save changes to a new csv

In [1996]:
standings_df.to_csv(os.path.join(data_folder, 'clean_full_standings.csv'), index=False)

# Transfers Preprocessing

In [1997]:
transfers_df.head()

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_euro,transfer_fee_million,...,to.club_name,to.competition_id,to.competition_name,date_of_birth_datetime,citizenship,position.main,position.other,height_cm,foot,updated_at
0,2518704,2019-07-01,free loan transfer,19/20,2019,314245,Emmanuel Iyoha,21.0,NaN,NaN,...,Holstein Kiel,L2,2. Bundesliga,1997-10-11,['Germany' 'Nigeria'],Left-Back,['Right-Back' 'Left Winger'],191.0,right,2024-02-25T02:16:17.122733
1,2472611,2019-07-01,free transfer,19/20,2019,153634,Marcel Hilßner,24.0,0.0,0.0,...,SC Paderborn,L1,Bundesliga,1995-01-30,['Germany'],Attacking Midfield,['Right Winger' 'Left Winger'],183.0,left,2024-02-24T10:13:57.036699
2,2512548,2019-07-01,free loan transfer,19/20,2019,195676,Ole Käuper,22.0,NaN,NaN,...,Carl Zeiss Jena,L3,3. Liga,1997-01-09,['Germany'],Defensive Midfield,['Central Midfield'],187.0,left,2024-02-24T10:13:48.186890
3,2507880,2019-07-01,free loan transfer,19/20,2019,236986,Thore Jacobsen,22.0,NaN,NaN,...,Magdeburg,L3,3. Liga,1997-04-19,['Germany'],Defensive Midfield,['Central Midfield' 'Left-Back'],182.0,left,2024-02-24T10:13:43.016587
4,2491956,2019-07-01,NaN,19/20,2019,31652,Fabian Schönheim,32.0,NaN,NaN,...,Without Club,NaN,NaN,1987-02-14,['Germany'],Centre-Back,['Left-Back' 'Defensive Midfield'],NaN,NaN,2024-02-24T23:49:23.092466


In [1998]:
transfers_df.shape

(2831, 27)

In [1999]:
transfers_df.dtypes

transfer_id                 int64
transfer_datetime          object
transfer_type              object
season                     object
season_id                   int64
player_id                   int64
player_name                object
player_age_at_transfer    float64
transfer_fee_euro         float64
transfer_fee_million      float64
market_value_euro         float64
market_value_million      float64
from.club_id                int64
from.club_name             object
from.competition_id        object
from.competition_name      object
to.club_id                  int64
to.club_name               object
to.competition_id          object
to.competition_name        object
date_of_birth_datetime     object
citizenship                object
position.main              object
position.other             object
height_cm                 float64
foot                       object
updated_at                 object
dtype: object

In [2000]:
transfers_df.describe()

,transfer_id,season_id,player_id,player_age_at_transfer,transfer_fee_euro,transfer_fee_million,market_value_euro,market_value_million,from.club_id,to.club_id,height_cm
count,2.831000e+03,2831.000000,2.831000e+03,2830.000000,1.220000e+03,1220.000000,2.688000e+03,2688.000000,2831.000000,2831.000000,2812.000000
mean,3.581345e+06,2021.080184,3.485626e+05,23.598940,4.697951e+06,4.697951,4.862029e+06,4.862029,1965.104557,2105.504062,183.764225
std,7.165994e+05,1.421292,1.954516e+05,4.118508,1.079437e+07,10.794374,9.649532e+06,9.649532,6038.719324,7408.450531,6.475927
min,2.095491e+06,2019.000000,5.320000e+02,15.000000,0.000000e+00,0.000000,1.000000e+04,0.010000,1.000000,1.000000,164.000000
25%,2.999282e+06,2020.000000,1.964210e+05,20.000000,0.000000e+00,0.000000,5.000000e+05,0.500000,33.000000,33.000000,179.000000
50%,3.525747e+06,2021.000000,3.380320e+05,23.000000,5.000000e+05,0.500000,1.600000e+06,1.600000,87.000000,86.000000,184.000000
75%,4.203492e+06,2022.000000,4.674370e+05,26.000000,4.000000e+06,4.000000,5.000000e+06,5.000000,533.000000,515.000000,188.000000
max,4.892349e+06,2023.000000,1.073997e+06,41.000000,1.030000e+08,103.000000,1.500000e+08,150.000000,51931.000000,87164.000000,202.000000


## Column names

In [2001]:
transfers_df.columns

Index(['transfer_id', 'transfer_datetime', 'transfer_type', 'season',
       'season_id', 'player_id', 'player_name', 'player_age_at_transfer',
       'transfer_fee_euro', 'transfer_fee_million', 'market_value_euro',
       'market_value_million', 'from.club_id', 'from.club_name',
       'from.competition_id', 'from.competition_name', 'to.club_id',
       'to.club_name', 'to.competition_id', 'to.competition_name',
       'date_of_birth_datetime', 'citizenship', 'position.main',
       'position.other', 'height_cm', 'foot', 'updated_at'],
      dtype='object')

In [2002]:
transfers_df.columns = [col.lower().replace('.', '_') for col in transfers_df.columns]

In [2003]:
transfers_df.columns

Index(['transfer_id', 'transfer_datetime', 'transfer_type', 'season',
       'season_id', 'player_id', 'player_name', 'player_age_at_transfer',
       'transfer_fee_euro', 'transfer_fee_million', 'market_value_euro',
       'market_value_million', 'from_club_id', 'from_club_name',
       'from_competition_id', 'from_competition_name', 'to_club_id',
       'to_club_name', 'to_competition_id', 'to_competition_name',
       'date_of_birth_datetime', 'citizenship', 'position_main',
       'position_other', 'height_cm', 'foot', 'updated_at'],
      dtype='object')

## Clean unnecessary columns

drop euros -> stay in millions

In [2004]:
transfers_df.drop(['transfer_fee_euro', 'market_value_euro'], axis=1, inplace=True)

In [2005]:
transfers_df.columns

Index(['transfer_id', 'transfer_datetime', 'transfer_type', 'season',
       'season_id', 'player_id', 'player_name', 'player_age_at_transfer',
       'transfer_fee_million', 'market_value_million', 'from_club_id',
       'from_club_name', 'from_competition_id', 'from_competition_name',
       'to_club_id', 'to_club_name', 'to_competition_id',
       'to_competition_name', 'date_of_birth_datetime', 'citizenship',
       'position_main', 'position_other', 'height_cm', 'foot', 'updated_at'],
      dtype='object')

## Transfer types

### Types

In [2006]:
transfers_df['transfer_type'].unique()

array(['free loan transfer', 'free transfer', nan, 'paid transfer',
       'unknown', 'paid loan transfer', 'end of loan'], dtype=object)

### Nans

In [2007]:
nan_count = pd.DataFrame(transfers_df.isna().sum()).T
nan_count.loc[:, nan_count.iloc[0] != 0]

,transfer_type,player_age_at_transfer,transfer_fee_million,market_value_million,from_competition_id,from_competition_name,to_competition_id,to_competition_name,date_of_birth_datetime,position_other,height_cm,foot
0,347,1,1611,143,225,225,193,193,1,596,19,70


In [2008]:
transfers_df[transfers_df['transfer_type'].isna()]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at
4,2491956,2019-07-01,NaN,19/20,2019,31652,Fabian Schönheim,32.0,NaN,0.20,...,Without Club,NaN,NaN,1987-02-14,['Germany'],Centre-Back,['Left-Back' 'Defensive Midfield'],NaN,NaN,2024-02-24T23:49:23.092466
6,2479748,2019-07-01,NaN,19/20,2019,1662,Sascha Riether,36.0,NaN,NaN,...,Retired,NaN,NaN,1983-03-23,['Germany'],Right-Back,['Defensive Midfield' 'Right Midfield'],NaN,NaN,2024-02-24T10:21:51.653669
9,2478605,2019-07-01,NaN,19/20,2019,428189,Tomáš Ostrák,19.0,NaN,0.10,...,1.FC Köln,L1,Bundesliga,2000-02-05,['Czech Republic'],Attacking Midfield,['Right Winger' 'Left Winger'],176.0,right,2024-02-25T16:35:20.025529
14,2475741,2019-07-01,NaN,19/20,2019,14977,Christoph Janker,34.0,NaN,0.10,...,Retired,NaN,NaN,1985-02-14,['Germany'],Centre-Back,['Right-Back' 'Defensive Midfield'],185.0,right,2024-02-24T23:58:02.661245
15,2686865,2019-07-01,NaN,19/20,2019,3851,Jan-Ingwer Callsen-Bracker,34.0,NaN,NaN,...,Retired,NaN,NaN,1984-09-23,['Germany'],Centre-Back,['Defensive Midfield'],188.0,right,2024-02-24T23:58:01.873945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606,4871664,2024-01-01,NaN,23/24,2023,720162,Justin Diehl,19.0,NaN,0.50,...,1.FC Köln,L1,Bundesliga,2004-11-27,['Germany' 'Ghana'],Left Winger,['Second Striker' 'Centre-Forward'],174.0,right,2024-02-26T12:32:51.101035
2610,4846063,2024-01-01,NaN,23/24,2023,923595,Max Moerstedt,17.0,NaN,NaN,...,TSG Hoffenheim,L1,Bundesliga,2006-01-15,['Germany'],Centre-Forward,NaN,194.0,right,2024-02-26T15:11:48.922731
2627,4811274,2024-01-09,NaN,23/24,2023,291422,Lys Mousset,27.0,NaN,0.40,...,Without Club,NaN,NaN,1996-02-08,['France' 'Senegal'],Centre-Forward,NaN,186.0,right,2024-02-24T23:23:22.673174
2636,4819471,2024-01-12,NaN,23/24,2023,908951,Noah Engelbreth,18.0,NaN,NaN,...,Union Berlin,L1,Bundesliga,2005-06-12,['Germany'],Defensive Midfield,['Centre-Back'],NaN,NaN,2024-02-26T04:01:07.467071


#### Transfers that are null and ended in retired

In [2009]:
print(f"Transfer types as null -> ended as Retired: {transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['to_club_name'] == 'Retired')].shape[0]}")
print(f"Transfer ended as Retired: {transfers_df[(transfers_df['to_club_name'] == 'Retired')].shape[0]}")

Transfer types as null -> ended as Retired: 33
Transfer ended as Retired: 33


In [2010]:
# Change these nans per transfer retirement
transfers_df.loc[transfers_df['to_club_name'] == 'Retired', 'transfer_type'] = 'transfer retirement'
transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['to_club_name'] == 'Retired')].shape[0]

0

In [2011]:
# Rest of the nan values
transfers_df[transfers_df['transfer_type'].isna()]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at
4,2491956,2019-07-01,NaN,19/20,2019,31652,Fabian Schönheim,32.0,NaN,0.20,...,Without Club,NaN,NaN,1987-02-14,['Germany'],Centre-Back,['Left-Back' 'Defensive Midfield'],NaN,NaN,2024-02-24T23:49:23.092466
9,2478605,2019-07-01,NaN,19/20,2019,428189,Tomáš Ostrák,19.0,NaN,0.10,...,1.FC Köln,L1,Bundesliga,2000-02-05,['Czech Republic'],Attacking Midfield,['Right Winger' 'Left Winger'],176.0,right,2024-02-25T16:35:20.025529
17,2809956,2019-07-01,NaN,19/20,2019,346329,Georgios Siadas,19.0,NaN,0.20,...,Düsseldorf II,NaN,NaN,1999-12-27,['Germany' 'Greece'],Left-Back,['Left Winger'],180.0,left,2024-02-25T17:47:31.367229
18,2473750,2019-07-01,NaN,19/20,2019,244945,Jannick Theißen,21.0,NaN,0.10,...,F. Düsseldorf,L1,Bundesliga,1998-01-07,['Germany'],Goalkeeper,NaN,188.0,right,2024-02-25T17:47:28.352573
19,2561419,2019-07-01,NaN,19/20,2019,380273,Felix Schwarzholz,19.0,NaN,0.10,...,FC Augsburg II,NaN,NaN,1999-09-24,['Germany'],Defensive Midfield,['Right-Back' 'Left-Back'],181.0,right,2024-02-25T17:09:48.795887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2606,4871664,2024-01-01,NaN,23/24,2023,720162,Justin Diehl,19.0,NaN,0.50,...,1.FC Köln,L1,Bundesliga,2004-11-27,['Germany' 'Ghana'],Left Winger,['Second Striker' 'Centre-Forward'],174.0,right,2024-02-26T12:32:51.101035
2610,4846063,2024-01-01,NaN,23/24,2023,923595,Max Moerstedt,17.0,NaN,NaN,...,TSG Hoffenheim,L1,Bundesliga,2006-01-15,['Germany'],Centre-Forward,NaN,194.0,right,2024-02-26T15:11:48.922731
2627,4811274,2024-01-09,NaN,23/24,2023,291422,Lys Mousset,27.0,NaN,0.40,...,Without Club,NaN,NaN,1996-02-08,['France' 'Senegal'],Centre-Forward,NaN,186.0,right,2024-02-24T23:23:22.673174
2636,4819471,2024-01-12,NaN,23/24,2023,908951,Noah Engelbreth,18.0,NaN,NaN,...,Union Berlin,L1,Bundesliga,2005-06-12,['Germany'],Defensive Midfield,['Centre-Back'],NaN,NaN,2024-02-26T04:01:07.467071


#### Transfers that are null and ended without club

In [2012]:
print(f"Transfer types as null -> ended as Without Club: {transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['to_club_name'] == 'Without Club')].shape[0]}")
print(f"Transfer ended as Without Club: {transfers_df[(transfers_df['to_club_name'] == 'Without Club')].shape[0]}")

Transfer types as null -> ended as Without Club: 55
Transfer ended as Without Club: 55


In [2013]:
# Change these nans per transfer to no club
transfers_df.loc[transfers_df['to_club_name'] == 'Without Club', 'transfer_type'] = 'transfer to no club'
transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['to_club_name'] == 'Without Club')].shape[0]

0

In [2014]:
# Rest of the nan values
transfers_df[transfers_df['transfer_type'].isna()]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at
9,2478605,2019-07-01,NaN,19/20,2019,428189,Tomáš Ostrák,19.0,NaN,0.10,...,1.FC Köln,L1,Bundesliga,2000-02-05,['Czech Republic'],Attacking Midfield,['Right Winger' 'Left Winger'],176.0,right,2024-02-25T16:35:20.025529
17,2809956,2019-07-01,NaN,19/20,2019,346329,Georgios Siadas,19.0,NaN,0.20,...,Düsseldorf II,NaN,NaN,1999-12-27,['Germany' 'Greece'],Left-Back,['Left Winger'],180.0,left,2024-02-25T17:47:31.367229
18,2473750,2019-07-01,NaN,19/20,2019,244945,Jannick Theißen,21.0,NaN,0.10,...,F. Düsseldorf,L1,Bundesliga,1998-01-07,['Germany'],Goalkeeper,NaN,188.0,right,2024-02-25T17:47:28.352573
19,2561419,2019-07-01,NaN,19/20,2019,380273,Felix Schwarzholz,19.0,NaN,0.10,...,FC Augsburg II,NaN,NaN,1999-09-24,['Germany'],Defensive Midfield,['Right-Back' 'Left-Back'],181.0,right,2024-02-25T17:09:48.795887
21,2506163,2019-07-01,NaN,19/20,2019,469949,Noah Katterbach,18.0,NaN,0.10,...,1.FC Köln,L1,Bundesliga,2001-04-13,['Germany'],Left-Back,['Left Midfield' 'Right-Back'],180.0,left,2024-02-25T16:35:19.375399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597,4711961,2023-11-01,NaN,23/24,2023,792380,Aleksandar Pavlovic,19.0,NaN,0.20,...,Bayern Munich,L1,Bundesliga,2004-05-03,['Germany' 'Serbia'],Defensive Midfield,['Central Midfield'],188.0,both,2024-02-26T09:43:15.623178
2606,4871664,2024-01-01,NaN,23/24,2023,720162,Justin Diehl,19.0,NaN,0.50,...,1.FC Köln,L1,Bundesliga,2004-11-27,['Germany' 'Ghana'],Left Winger,['Second Striker' 'Centre-Forward'],174.0,right,2024-02-26T12:32:51.101035
2610,4846063,2024-01-01,NaN,23/24,2023,923595,Max Moerstedt,17.0,NaN,NaN,...,TSG Hoffenheim,L1,Bundesliga,2006-01-15,['Germany'],Centre-Forward,NaN,194.0,right,2024-02-26T15:11:48.922731
2636,4819471,2024-01-12,NaN,23/24,2023,908951,Noah Engelbreth,18.0,NaN,NaN,...,Union Berlin,L1,Bundesliga,2005-06-12,['Germany'],Defensive Midfield,['Centre-Back'],NaN,NaN,2024-02-26T04:01:07.467071


#### Transfers that are null and went to a club comming from no club

In [2015]:
print(f"Transfer types as null -> comming Without Club: {transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['from_club_name'] == 'Without Club')].shape[0]}")
print(f"Transfer comming as Without Club: {transfers_df[(transfers_df['from_club_name'] == 'Without Club')].shape[0]}")

Transfer types as null -> comming Without Club: 9
Transfer comming as Without Club: 9


In [2016]:
# Change these nans per transfer to no club
transfers_df.loc[transfers_df['from_club_name'] == 'Without Club', 'transfer_type'] = 'transfer from no club'
transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['from_club_name'] == 'Without Club')].shape[0]

0

In [2017]:
# Rest of the nan values
transfers_df[transfers_df['transfer_type'].isna()]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at
9,2478605,2019-07-01,NaN,19/20,2019,428189,Tomáš Ostrák,19.0,NaN,0.10,...,1.FC Köln,L1,Bundesliga,2000-02-05,['Czech Republic'],Attacking Midfield,['Right Winger' 'Left Winger'],176.0,right,2024-02-25T16:35:20.025529
17,2809956,2019-07-01,NaN,19/20,2019,346329,Georgios Siadas,19.0,NaN,0.20,...,Düsseldorf II,NaN,NaN,1999-12-27,['Germany' 'Greece'],Left-Back,['Left Winger'],180.0,left,2024-02-25T17:47:31.367229
18,2473750,2019-07-01,NaN,19/20,2019,244945,Jannick Theißen,21.0,NaN,0.10,...,F. Düsseldorf,L1,Bundesliga,1998-01-07,['Germany'],Goalkeeper,NaN,188.0,right,2024-02-25T17:47:28.352573
19,2561419,2019-07-01,NaN,19/20,2019,380273,Felix Schwarzholz,19.0,NaN,0.10,...,FC Augsburg II,NaN,NaN,1999-09-24,['Germany'],Defensive Midfield,['Right-Back' 'Left-Back'],181.0,right,2024-02-25T17:09:48.795887
21,2506163,2019-07-01,NaN,19/20,2019,469949,Noah Katterbach,18.0,NaN,0.10,...,1.FC Köln,L1,Bundesliga,2001-04-13,['Germany'],Left-Back,['Left Midfield' 'Right-Back'],180.0,left,2024-02-25T16:35:19.375399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597,4711961,2023-11-01,NaN,23/24,2023,792380,Aleksandar Pavlovic,19.0,NaN,0.20,...,Bayern Munich,L1,Bundesliga,2004-05-03,['Germany' 'Serbia'],Defensive Midfield,['Central Midfield'],188.0,both,2024-02-26T09:43:15.623178
2606,4871664,2024-01-01,NaN,23/24,2023,720162,Justin Diehl,19.0,NaN,0.50,...,1.FC Köln,L1,Bundesliga,2004-11-27,['Germany' 'Ghana'],Left Winger,['Second Striker' 'Centre-Forward'],174.0,right,2024-02-26T12:32:51.101035
2610,4846063,2024-01-01,NaN,23/24,2023,923595,Max Moerstedt,17.0,NaN,NaN,...,TSG Hoffenheim,L1,Bundesliga,2006-01-15,['Germany'],Centre-Forward,NaN,194.0,right,2024-02-26T15:11:48.922731
2636,4819471,2024-01-12,NaN,23/24,2023,908951,Noah Engelbreth,18.0,NaN,NaN,...,Union Berlin,L1,Bundesliga,2005-06-12,['Germany'],Defensive Midfield,['Centre-Back'],NaN,NaN,2024-02-26T04:01:07.467071


#### Transfers on the same org

Transfers that happend in the same organization
- Going from B team to A team
- Going from A team to B team
- Going from under 19 to A team

In [2018]:
b_teams_str = ['U19', 'U17', 'II']
inner_transfer_types = ['transfer inner upgrade', 'transfer inner downgrade']

In [2019]:
for team_comming in b_teams_str:
    print(f"Going from {team_comming}: {transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['from_club_name'].str.contains(team_comming))].shape[0]}")
    print(f"Going to {team_comming}: {transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['to_club_name'].str.contains(team_comming))].shape[0]}")

Going from U19: 122
Going to U19: 0
Going from U17: 4
Going to U17: 0
Going from II: 91
Going to II: 32


In [2020]:
# Change these nans per transfer inner upgrade or transfer inner downgrade
for team_comming in b_teams_str:
    transfers_df.loc[transfers_df['to_club_name'].str.contains(team_comming), 'transfer_type'] = inner_transfer_types[0]
    transfers_df.loc[transfers_df['from_club_name'].str.contains(team_comming), 'transfer_type'] = inner_transfer_types[1]

In [2021]:
for team_comming in b_teams_str:
    print(f"Going from {team_comming}: {transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['from_club_name'].str.contains(team_comming))].shape[0]}")
    print(f"Going to {team_comming}: {transfers_df[(transfers_df['transfer_type'].isna()) & (transfers_df['to_club_name'].str.contains(team_comming))].shape[0]}")

Going from U19: 0
Going to U19: 0
Going from U17: 0
Going to U17: 0
Going from II: 0
Going to II: 0


In [2022]:
# Rest of the nan values
transfers_df[transfers_df['transfer_type'].isna()]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at
190,2538108,2019-07-04,NaN,19/20,2019,4360,Arjen Robben,35.0,NaN,NaN,...,Career break,NaN,NaN,1984-01-23,['Netherlands'],Right Winger,['Left Winger'],180.0,left,2024-02-25T01:27:37.300391


In [2023]:
transfers_df[transfers_df['player_id']==4360]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at
190,2538108,2019-07-04,NaN,19/20,2019,4360,Arjen Robben,35.0,NaN,NaN,...,Career break,NaN,NaN,1984-01-23,['Netherlands'],Right Winger,['Left Winger'],180.0,left,2024-02-25T01:27:37.300391


In [2024]:
# Change these nans per transfer career break
transfers_df.loc[transfers_df['to_club_name'] == 'Career break', 'transfer_type'] = 'transfer career break'

In [2025]:
# Rest of the nan values
transfers_df[transfers_df['transfer_type'].isna()]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at


### Final transfer types

In [2026]:
transfers_df['transfer_type'].unique()

array(['free loan transfer', 'free transfer', 'transfer to no club',
       'paid transfer', 'transfer retirement', 'transfer inner downgrade',
       'unknown', 'transfer inner upgrade', 'paid loan transfer',
       'transfer career break', 'end of loan', 'transfer from no club'],
      dtype=object)

## Season id

In [2027]:
transfers_df['season_id'].unique()

array([2019, 2020, 2021, 2022, 2023])

## Player id

In [2028]:
transfers_df['player_id'].nunique()

1263

## Player age

In [2029]:
transfers_df['player_age_at_transfer'].unique()

array([21., 24., 22., 32., 23., 36., 20., 19., 25., 28., 35., 34., 18.,
       29., 17., 33., 39., 26., 31., 27., 30., 37., 41., 15., nan, 16.,
       38.])

One nan value, let's replace the nan with tha actual value

In [2030]:
nan_count = pd.DataFrame(transfers_df.isna().sum()).T
nan_count.loc[:, nan_count.iloc[0] != 0]

,player_age_at_transfer,transfer_fee_million,market_value_million,from_competition_id,from_competition_name,to_competition_id,to_competition_name,date_of_birth_datetime,position_other,height_cm,foot
0,1,1611,143,225,225,193,193,1,596,19,70


In [2031]:
transfers_df[transfers_df['player_age_at_transfer'].isna()]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at
1146,3219683,2021-07-01,transfer retirement,21/22,2021,58124,Julian Schieber,NaN,NaN,NaN,...,Retired,NaN,NaN,NaN,['Germany'],Centre-Forward,['Second Striker' 'Left Winger'],186.0,left,2024-02-25T00:09:12.367468


In [2032]:
# value to replace
transfers_df[transfers_df['player_age_at_transfer'].isna()]['transfer_datetime'], transfers_df[transfers_df['player_age_at_transfer'].isna()]['date_of_birth_datetime']

(1146    2021-07-01
 Name: transfer_datetime, dtype: object,
 1146    NaN
 Name: date_of_birth_datetime, dtype: object)

In [2033]:
# as date_of_birth_datetime is nan, we search the value and the replace the nan
transfers_df.loc[transfers_df['player_age_at_transfer'].isna(), 'date_of_birth_datetime'] = '1989-02-13'

In [2034]:
transfers_df['player_age_at_transfer'].fillna(pd.to_datetime(transfers_df['transfer_datetime']).dt.year - pd.to_datetime(transfers_df['date_of_birth_datetime']).dt.year, inplace=True)

In [2035]:
transfers_df[transfers_df['player_id']==58124]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_club_name,to_competition_id,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at
1146,3219683,2021-07-01,transfer retirement,21/22,2021,58124,Julian Schieber,32.0,NaN,NaN,...,Retired,NaN,NaN,1989-02-13,['Germany'],Centre-Forward,['Second Striker' 'Left Winger'],186.0,left,2024-02-25T00:09:12.367468


## Club names

Taking into account club names of standings so we can join them

In [2036]:
print(f"Number of unique clubs ids: {transfers_df['from_club_id'].nunique()}")

Number of unique clubs ids: 400


### From

In [2037]:
result = transfers_df.groupby('from_club_id')['from_club_name'].nunique().reset_index().sort_values('from_club_name', ascending=False)
result

,from_club_id,from_club_name
394,41101,2
0,1,1
262,1268,1
272,1464,1
271,1463,1
...,...,...
130,289,1
129,288,1
128,281,1
127,279,1


In [2038]:
transfers_df[transfers_df['from_club_id']==41101][['from_club_id', 'from_club_name']]

,from_club_id,from_club_name
849,41101,Juventus U23
1138,41101,Juventus U23
2720,41101,Juve Next Gen


In [2039]:
transfers_df.loc[transfers_df['from_club_id']==41101, 'from_club_name'] = 'Juventus U23'
transfers_df[transfers_df['from_club_id']==41101][['from_club_id', 'from_club_name']]

,from_club_id,from_club_name
849,41101,Juventus U23
1138,41101,Juventus U23
2720,41101,Juventus U23


### To

In [2040]:
result = transfers_df.groupby('to_club_id')['to_club_name'].nunique().reset_index().sort_values('to_club_name', ascending=False)
result

,to_club_id,to_club_name
72,121,2
416,41101,2
0,1,1
281,1242,1
291,1465,1
...,...,...
140,350,1
139,347,1
138,337,1
137,336,1


In [2041]:
transfers_df[transfers_df['to_club_id']==41101][['to_club_id', 'to_club_name']]

,to_club_id,to_club_name
1005,41101,Juventus U23
2572,41101,Juve Next Gen


In [2042]:
transfers_df[transfers_df['to_club_id']==121][['to_club_id', 'to_club_name']]

,to_club_id,to_club_name
264,121,Dinamo Moscow
982,121,Dynamo Moscow
2744,121,Dynamo Moscow


In [2043]:
transfers_df[transfers_df['from_club_id']==121][['from_club_id', 'from_club_name']]

,from_club_id,from_club_name
817,121,Dynamo Moscow
1128,121,Dynamo Moscow
1823,121,Dynamo Moscow


In [2044]:
transfers_df.loc[transfers_df['to_club_id']==41101, 'to_club_name'] = 'Juventus U23'
transfers_df.loc[transfers_df['to_club_id']==121, 'to_club_name'] = 'Dynamo Moscow'

In [2045]:
result = transfers_df.groupby('to_club_id')['to_club_name'].nunique().reset_index().sort_values('to_club_name', ascending=False)
result

,to_club_id,to_club_name
0,1,1
293,1468,1
291,1465,1
290,1426,1
289,1421,1
...,...,...
139,347,1
138,337,1
137,336,1
136,331,1


### Standarize bundesliga team names to match Standings file to later cross

In [2046]:
list(set(list(transfers_df[transfers_df['from_competition_id']=='L1']['from_club_name'].unique()) + list(transfers_df[transfers_df['to_competition_id']=='L1']['to_club_name'].unique())))

['VfB Stuttgart',
 'VfL Bochum',
 'VfL Wolfsburg',
 '1.FSV Mainz 05',
 'SC Paderborn',
 'B. Leverkusen',
 'Greuther Fürth',
 'Darmstadt 98',
 'Bor. Dortmund',
 'TSG Hoffenheim',
 'Heidenheim',
 'Bayern Munich',
 'Hertha BSC',
 'Arm. Bielefeld',
 'FC Schalke 04',
 'RB Leipzig',
 'F. Düsseldorf',
 'Union Berlin',
 'Werder Bremen',
 'E. Frankfurt',
 'FC Augsburg',
 'SC Freiburg',
 '1.FC Köln',
 "Bor. M'gladbach"]

In [2047]:
list(standings_df['team_name'].unique())

['Bayern Munich',
 'Borussia Dortmund',
 'RB Leipzig',
 'Borussia Mönchengladbach',
 'Bayer 04 Leverkusen',
 '1899 Hoffenheim',
 'VfL Wolfsburg',
 'SC Freiburg',
 'Eintracht Frankfurt',
 'Hertha Berlin',
 'FC Union Berlin',
 'FC Schalke 04',
 'Mainz 05',
 'FC Köln',
 'FC Augsburg',
 'Werder Bremen',
 'Fortuna Düsseldorf',
 'SC Paderborn 07',
 'VfB Stuttgart',
 'Arminia Bielefeld',
 'VfL Bochum',
 'Greuther Fürth',
 'FC Heidenheim',
 'Darmstadt 98']

In [2048]:
d_team_name_correction_transfers = {
    '1.FSV Mainz 05': 'Mainz 05',
    'SC Paderborn': 'SC Paderborn 07',
    'B. Leverkusen': 'Bayer 04 Leverkusen',
    'Bor. Dortmund': 'Borussia Dortmund',
    'TSG Hoffenheim': '1899 Hoffenheim',
    'Heidenheim': 'FC Heidenheim',
    'Hertha BSC': 'Hertha Berlin',
    'Arm. Bielefeld': 'Arminia Bielefeld',
    'F. Düsseldorf': 'Fortuna Düsseldorf',
    'Union Berlin': 'FC Union Berlin',
    'E. Frankfurt': 'Eintracht Frankfurt',
    '1.FC Köln': 'FC Köln',
    "Bor. M'gladbach": 'Borussia Mönchengladbach'
}

transfers_df['from_club_name'] = transfers_df['from_club_name'].replace(d_team_name_correction_transfers)
transfers_df['to_club_name'] = transfers_df['to_club_name'].replace(d_team_name_correction_transfers)

In [2049]:
list(set(list(transfers_df[transfers_df['from_competition_id']=='L1']['from_club_name'].unique()) + list(transfers_df[transfers_df['to_competition_id']=='L1']['to_club_name'].unique())))

['VfB Stuttgart',
 'VfL Bochum',
 'VfL Wolfsburg',
 'Fortuna Düsseldorf',
 'Greuther Fürth',
 'Eintracht Frankfurt',
 'Darmstadt 98',
 'Borussia Mönchengladbach',
 'SC Paderborn 07',
 'Bayern Munich',
 'Hertha Berlin',
 'Arminia Bielefeld',
 'Bayer 04 Leverkusen',
 'Borussia Dortmund',
 'FC Köln',
 'FC Schalke 04',
 'RB Leipzig',
 'FC Union Berlin',
 'Werder Bremen',
 '1899 Hoffenheim',
 'FC Augsburg',
 'SC Freiburg',
 'Mainz 05',
 'FC Heidenheim']

## Positions

### main

In [2050]:
print(f"Unique positions: {transfers_df['position_main'].unique()}")
print(f"Number of unique positions: {transfers_df['position_main'].nunique()}")

Unique positions: ['Left-Back' 'Attacking Midfield' 'Defensive Midfield' 'Centre-Back'
 'Right-Back' 'Centre-Forward' 'Goalkeeper' 'Left Winger' 'Right Midfield'
 'Central Midfield' 'Right Winger' 'Left Midfield' 'Second Striker']
Number of unique positions: 13


In [2051]:
# lower and standarize all positions
transfers_df['position_main'] = transfers_df['position_main'].str.lower().str.replace(' ', '-')

In [2052]:
transfers_df['position_main'].unique()

array(['left-back', 'attacking-midfield', 'defensive-midfield',
       'centre-back', 'right-back', 'centre-forward', 'goalkeeper',
       'left-winger', 'right-midfield', 'central-midfield',
       'right-winger', 'left-midfield', 'second-striker'], dtype=object)

### other

In [2053]:
print(f"Unique positions: {transfers_df['position_other'].unique()}")
print(f"Number of unique positions: {transfers_df['position_other'].nunique()}")

Unique positions: ["['Right-Back' 'Left Winger']" "['Right Winger' 'Left Winger']"
 "['Central Midfield']" "['Central Midfield' 'Left-Back']"
 "['Left-Back' 'Defensive Midfield']" "['Right-Back']"
 "['Defensive Midfield' 'Right Midfield']"
 "['Right-Back' 'Defensive Midfield']" nan
 "['Left Winger' 'Right Winger']" "['Second Striker' 'Left Winger']"
 "['Right Midfield' 'Defensive Midfield']" "['Defensive Midfield']"
 "['Left Winger']" "['Right-Back' 'Left-Back']"
 "['Left Midfield' 'Right-Back']" "['Right Winger' 'Left-Back']"
 "['Right Midfield']" "['Attacking Midfield' 'Right Midfield']"
 "['Attacking Midfield' 'Defensive Midfield']"
 "['Left Winger' 'Right-Back']" "['Central Midfield' 'Centre-Back']"
 "['Right-Back' 'Right Midfield']"
 "['Central Midfield' 'Attacking Midfield']"
 "['Attacking Midfield' 'Centre-Forward']"
 "['Centre-Back' 'Defensive Midfield']" "['Left Midfield' 'Left Winger']"
 "['Centre-Back' 'Left Midfield']" "['Right Winger' 'Left Midfield']"
 "['Right Midfield' 

In [2054]:
# replace nan with empty list
transfers_df['position_other'].fillna('[]', inplace=True)

In [2055]:
def split_list_column(data_point):
    if len(data_point) > 0:
        x = [d for d in data_point.lstrip('[').rstrip(']').split("'") if d and len(d)>1]
    else:
        x = []
    return x

transfers_df['position_other_processed'] = transfers_df['position_other'].apply(split_list_column)
transfers_df['position_other_processed']

0                [Right-Back, Left Winger]
1              [Right Winger, Left Winger]
2                       [Central Midfield]
3            [Central Midfield, Left-Back]
4          [Left-Back, Defensive Midfield]
                       ...                
2826           [Left Winger, Right Winger]
2827                         [Centre-Back]
2828                       [Left Midfield]
2829           [Left Midfield, Right-Back]
2830    [Right Winger, Attacking Midfield]
Name: position_other_processed, Length: 2831, dtype: object

In [2056]:
values_expanded = transfers_df['position_other_processed'].apply(pd.Series)
values_expanded.columns = [f'extra_position_{i+1}' for i in range(values_expanded.shape[1])]
values_expanded

,extra_position_1,extra_position_2
0,Right-Back,Left Winger
1,Right Winger,Left Winger
2,Central Midfield,NaN
3,Central Midfield,Left-Back
4,Left-Back,Defensive Midfield
...,...,...
2826,Left Winger,Right Winger
2827,Centre-Back,NaN
2828,Left Midfield,NaN
2829,Left Midfield,Right-Back


In [2057]:
values_expanded['extra_position_1'] = values_expanded['extra_position_1'].str.lower().str.replace(' ', '-')
values_expanded['extra_position_2'] = values_expanded['extra_position_2'].str.lower().str.replace(' ', '-')

In [2058]:
transfers_df = transfers_df.drop('position_other_processed', axis=1).join(values_expanded)
transfers_df

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at,extra_position_1,extra_position_2
0,2518704,2019-07-01,free loan transfer,19/20,2019,314245,Emmanuel Iyoha,21.0,NaN,0.90,...,2. Bundesliga,1997-10-11,['Germany' 'Nigeria'],left-back,['Right-Back' 'Left Winger'],191.0,right,2024-02-25T02:16:17.122733,right-back,left-winger
1,2472611,2019-07-01,free transfer,19/20,2019,153634,Marcel Hilßner,24.0,0.0,0.40,...,Bundesliga,1995-01-30,['Germany'],attacking-midfield,['Right Winger' 'Left Winger'],183.0,left,2024-02-24T10:13:57.036699,right-winger,left-winger
2,2512548,2019-07-01,free loan transfer,19/20,2019,195676,Ole Käuper,22.0,NaN,0.35,...,3. Liga,1997-01-09,['Germany'],defensive-midfield,['Central Midfield'],187.0,left,2024-02-24T10:13:48.186890,central-midfield,NaN
3,2507880,2019-07-01,free loan transfer,19/20,2019,236986,Thore Jacobsen,22.0,NaN,0.30,...,3. Liga,1997-04-19,['Germany'],defensive-midfield,['Central Midfield' 'Left-Back'],182.0,left,2024-02-24T10:13:43.016587,central-midfield,left-back
4,2491956,2019-07-01,transfer to no club,19/20,2019,31652,Fabian Schönheim,32.0,NaN,0.20,...,NaN,1987-02-14,['Germany'],centre-back,['Left-Back' 'Defensive Midfield'],NaN,NaN,2024-02-24T23:49:23.092466,left-back,defensive-midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,4620327,2024-06-30,end of loan,23/24,2023,524290,Fisnik Asllani,21.0,NaN,0.80,...,Bundesliga,2002-08-08,['Germany' 'Kosovo'],centre-forward,['Left Winger' 'Right Winger'],191.0,both,2024-02-25T15:19:52.966974,left-winger,right-winger
2827,4865315,2024-06-30,end of loan,23/24,2023,485965,Nicolas Cozza,25.0,NaN,3.50,...,Bundesliga,1999-01-08,['France' 'Italy'],left-back,['Centre-Back'],178.0,left,2024-02-25T08:40:40.869712,centre-back,NaN
2828,4458245,2024-06-30,end of loan,23/24,2023,798043,Tom Rothe,19.0,NaN,6.00,...,Bundesliga,2004-10-29,['Germany'],left-back,['Left Midfield'],193.0,left,2024-02-25T23:43:07.687766,left-midfield,NaN
2829,4435369,2024-06-30,end of loan,23/24,2023,717346,Sanoussy Ba,20.0,NaN,0.50,...,Bundesliga,2004-01-05,['Germany' 'Senegal'],left-back,['Left Midfield' 'Right-Back'],184.0,both,2024-02-25T18:57:41.792456,left-midfield,right-back


In [2059]:
transfers_df['extra_position_1'].unique(), len(transfers_df['extra_position_1'].unique())

(array(['right-back', 'right-winger', 'central-midfield', 'left-back',
        'defensive-midfield', nan, 'left-winger', 'second-striker',
        'right-midfield', 'left-midfield', 'attacking-midfield',
        'centre-back', 'centre-forward'], dtype=object),
 13)

In [2060]:
transfers_df['extra_position_2'].unique(), len(transfers_df['extra_position_2'].unique())

(array(['left-winger', nan, 'left-back', 'defensive-midfield',
        'right-midfield', 'right-winger', 'right-back', 'centre-back',
        'attacking-midfield', 'centre-forward', 'left-midfield',
        'second-striker', 'central-midfield', 'sweeper'], dtype=object),
 14)

In [2061]:
transfers_df[transfers_df['extra_position_2']=='sweeper']

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,to_competition_name,date_of_birth_datetime,citizenship,position_main,position_other,height_cm,foot,updated_at,extra_position_1,extra_position_2
2051,4243623,2023-01-26,paid loan transfer,22/23,2022,460245,Moritz Jenz,23.0,0.2,4.0,...,Bundesliga,1999-04-30,['Germany'],centre-back,['Right-Back' 'Sweeper'],190.0,right,2024-02-25T03:08:55.464395,right-back,sweeper
2220,4243624,2023-06-30,end of loan,22/23,2022,460245,Moritz Jenz,24.0,NaN,7.5,...,Ligue 1,1999-04-30,['Germany'],centre-back,['Right-Back' 'Sweeper'],190.0,right,2024-02-25T03:08:55.464395,right-back,sweeper
2294,4440326,2023-07-01,paid transfer,23/24,2023,460245,Moritz Jenz,24.0,8.0,7.5,...,Bundesliga,1999-04-30,['Germany'],centre-back,['Right-Back' 'Sweeper'],190.0,right,2024-02-25T03:08:55.464395,right-back,sweeper


### Unique positions

In [2062]:
unique_positions = list(set(list(transfers_df['position_main'].unique()) + list(transfers_df['extra_position_1'].unique()) + list(transfers_df['extra_position_2'].unique())))
unique_positions = [x for x in unique_positions if str(x) != 'nan']
print(f"Final unique positions: {unique_positions}")
print(f"Total unique positions: {len(unique_positions)}")

Final unique positions: ['right-winger', 'defensive-midfield', 'left-midfield', 'goalkeeper', 'left-winger', 'centre-forward', 'right-midfield', 'attacking-midfield', 'centre-back', 'left-back', 'second-striker', 'right-back', 'sweeper', 'central-midfield']
Total unique positions: 14


### Group positions in goalkeeper, attack, mid, defense

In [2063]:
grouped_positions = {
    'goalkeeper': ['goalkeeper'],
    'defense': ['centre-back', 'left-back', 'right-back'],
    'mid': ['defensive-midfield', 'left-midfield', 'right-midfield', 'attacking-midfield', 'sweeper', 'central-midfield'],
    'attack': ['right-winger', 'left-winger', 'centre-forward', 'second-striker']
}

value_to_category = {value: category for category, values in grouped_positions.items() for value in values}


transfers_df['position_main_segment'] = transfers_df['position_main'].map(value_to_category)
transfers_df['extra_position_1_segment'] = transfers_df['extra_position_1'].map(value_to_category)
transfers_df['extra_position_2_segment'] = transfers_df['extra_position_2'].map(value_to_category)

## Foot

In [2064]:
transfers_df['foot'].unique()

array(['right', 'left', nan, 'both'], dtype=object)

In [2065]:
# replace nan with unknown
transfers_df['foot'].fillna('unknown', inplace=True)

In [2066]:
transfers_df['foot'].unique()

array(['right', 'left', 'unknown', 'both'], dtype=object)

## Citizenship

In [2067]:
# Same processing as positions other
print(f"Unique citizenship: {transfers_df['citizenship'].unique()}")
print(f"Number of unique citizenship: {transfers_df['citizenship'].nunique()}")

Unique citizenship: ["['Germany' 'Nigeria']" "['Germany']" "['Nigeria']" "['Czech Republic']"
 "['Spain']" "['Korea, South']" "['Netherlands']" "['Germany' 'Greece']"
 "['Belgium' 'DR Congo']" "['Austria']" "['Germany' 'DR Congo']"
 "['Switzerland']" "['Bulgaria' 'Germany']" "['Germany' 'Cameroon']"
 "['Serbia']" "['Guinea' 'France']" "['Philippines' 'Germany']"
 "['England']" "['Türkiye' 'Germany']" "['Bosnia-Herzegovina']"
 "['France']" "['Germany' 'Türkiye']" "['Poland' 'Germany']" "['Ghana']"
 "['Brazil']" "['Sweden' 'Eritrea']" "['Belgium']" "['Ecuador' 'Germany']"
 "['Russia']" "['Suriname' 'Netherlands']" "['Netherlands' 'Nigeria']"
 "['Italy']" "['Canada' 'Germany']" "['Israel' 'Portugal']"
 "['United States' 'Germany']" "['Ghana' 'Bulgaria']"
 "['Nigeria' 'Germany']" "['France' 'Martinique']" "['Ghana' 'Germany']"
 "['Japan']" "['Türkiye']" "['Brazil' 'Germany']"
 "['Switzerland' 'DR Congo']" "['Denmark']" "['Croatia' 'Germany']"
 "['Switzerland' 'Senegal']" "['Armenia']" "['S

In [2068]:
transfers_df['citizenship_processed'] = transfers_df['citizenship'].apply(split_list_column)
transfers_df['citizenship_processed']

0       [Germany, Nigeria]
1                [Germany]
2                [Germany]
3                [Germany]
4                [Germany]
               ...        
2826     [Germany, Kosovo]
2827       [France, Italy]
2828             [Germany]
2829    [Germany, Senegal]
2830               [Spain]
Name: citizenship_processed, Length: 2831, dtype: object

In [2069]:
citizenship_expanded = transfers_df['citizenship_processed'].apply(pd.Series)
citizenship_expanded.columns = [f'citizenship_{i+1}' for i in range(citizenship_expanded.shape[1])]
citizenship_expanded

,citizenship_1,citizenship_2,citizenship_3
0,Germany,Nigeria,NaN
1,Germany,NaN,NaN
2,Germany,NaN,NaN
3,Germany,NaN,NaN
4,Germany,NaN,NaN
...,...,...,...
2826,Germany,Kosovo,NaN
2827,France,Italy,NaN
2828,Germany,NaN,NaN
2829,Germany,Senegal,NaN


In [2070]:
transfers_df = transfers_df.drop('citizenship_processed', axis=1).join(citizenship_expanded)
transfers_df

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,foot,updated_at,extra_position_1,extra_position_2,position_main_segment,extra_position_1_segment,extra_position_2_segment,citizenship_1,citizenship_2,citizenship_3
0,2518704,2019-07-01,free loan transfer,19/20,2019,314245,Emmanuel Iyoha,21.0,NaN,0.90,...,right,2024-02-25T02:16:17.122733,right-back,left-winger,defense,defense,attack,Germany,Nigeria,NaN
1,2472611,2019-07-01,free transfer,19/20,2019,153634,Marcel Hilßner,24.0,0.0,0.40,...,left,2024-02-24T10:13:57.036699,right-winger,left-winger,mid,attack,attack,Germany,NaN,NaN
2,2512548,2019-07-01,free loan transfer,19/20,2019,195676,Ole Käuper,22.0,NaN,0.35,...,left,2024-02-24T10:13:48.186890,central-midfield,NaN,mid,mid,NaN,Germany,NaN,NaN
3,2507880,2019-07-01,free loan transfer,19/20,2019,236986,Thore Jacobsen,22.0,NaN,0.30,...,left,2024-02-24T10:13:43.016587,central-midfield,left-back,mid,mid,defense,Germany,NaN,NaN
4,2491956,2019-07-01,transfer to no club,19/20,2019,31652,Fabian Schönheim,32.0,NaN,0.20,...,unknown,2024-02-24T23:49:23.092466,left-back,defensive-midfield,defense,defense,mid,Germany,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,4620327,2024-06-30,end of loan,23/24,2023,524290,Fisnik Asllani,21.0,NaN,0.80,...,both,2024-02-25T15:19:52.966974,left-winger,right-winger,attack,attack,attack,Germany,Kosovo,NaN
2827,4865315,2024-06-30,end of loan,23/24,2023,485965,Nicolas Cozza,25.0,NaN,3.50,...,left,2024-02-25T08:40:40.869712,centre-back,NaN,defense,defense,NaN,France,Italy,NaN
2828,4458245,2024-06-30,end of loan,23/24,2023,798043,Tom Rothe,19.0,NaN,6.00,...,left,2024-02-25T23:43:07.687766,left-midfield,NaN,defense,mid,NaN,Germany,NaN,NaN
2829,4435369,2024-06-30,end of loan,23/24,2023,717346,Sanoussy Ba,20.0,NaN,0.50,...,both,2024-02-25T18:57:41.792456,left-midfield,right-back,defense,mid,defense,Germany,Senegal,NaN


## height_cm

In [2071]:
transfers_df['height_cm'].unique()

array([191., 183., 187., 182.,  nan, 193., 184., 185., 176., 190., 175.,
       188., 192., 180., 181., 194., 177., 186., 179., 196., 189., 174.,
       195., 173., 170., 178., 172., 171., 197., 168., 199., 198., 169.,
       166., 200., 165., 202., 164.])

In [2072]:
transfers_df[transfers_df['height_cm'].isna()]

,transfer_id,transfer_datetime,transfer_type,season,season_id,player_id,player_name,player_age_at_transfer,transfer_fee_million,market_value_million,...,foot,updated_at,extra_position_1,extra_position_2,position_main_segment,extra_position_1_segment,extra_position_2_segment,citizenship_1,citizenship_2,citizenship_3
4,2491956,2019-07-01,transfer to no club,19/20,2019,31652,Fabian Schönheim,32.0,NaN,0.200,...,unknown,2024-02-24T23:49:23.092466,left-back,defensive-midfield,defense,defense,mid,Germany,NaN,NaN
6,2479748,2019-07-01,transfer retirement,19/20,2019,1662,Sascha Riether,36.0,NaN,NaN,...,unknown,2024-02-24T10:21:51.653669,defensive-midfield,right-midfield,defense,mid,mid,Germany,NaN,NaN
13,2496839,2019-07-01,free transfer,19/20,2019,18021,Paul Verhaegh,35.0,0.0,0.750,...,unknown,2024-02-24T23:58:05.227858,right-midfield,defensive-midfield,defense,mid,mid,Netherlands,NaN,NaN
30,3728967,2019-07-01,transfer retirement,19/20,2019,160934,Leon Fesser,24.0,NaN,0.100,...,unknown,2024-02-25T03:27:18.356316,defensive-midfield,NaN,defense,mid,NaN,Germany,NaN,NaN
46,3222165,2019-07-01,transfer retirement,19/20,2019,38266,Marc Torrejón,33.0,NaN,NaN,...,unknown,2024-02-24T08:48:41.241803,right-back,NaN,defense,defense,NaN,Spain,NaN,NaN
56,2465073,2019-07-01,transfer retirement,19/20,2019,1784,René Adler,34.0,NaN,NaN,...,unknown,2024-02-24T22:41:26.413466,NaN,NaN,goalkeeper,NaN,NaN,Germany,NaN,NaN
139,2520845,2019-07-01,transfer retirement,19/20,2019,1126,Matthias Lehmann,36.0,NaN,NaN,...,unknown,2024-02-24T21:23:37.100245,central-midfield,right-midfield,mid,mid,mid,Germany,NaN,NaN
242,2583549,2019-07-31,paid loan transfer,19/20,2019,58205,André Schürrle,28.0,1.0,12.000,...,unknown,2024-02-25T00:24:10.237261,right-winger,second-striker,attack,attack,attack,Germany,NaN,NaN
435,2583550,2020-06-30,end of loan,19/20,2019,58205,André Schürrle,29.0,NaN,4.500,...,unknown,2024-02-25T00:24:10.237261,right-winger,second-striker,attack,attack,attack,Germany,NaN,NaN
537,2948014,2020-07-01,transfer retirement,20/21,2020,94559,Nils Quaschner,26.0,NaN,NaN,...,unknown,2024-02-24T07:06:07.466612,right-winger,left-winger,attack,attack,attack,Germany,NaN,NaN


## Save changes to a new df

In [2073]:
transfers_df.to_csv(os.path.join(data_folder, 'clean_transfers.csv'), index=False)